In [55]:
import pandas as pd
import geopandas as gpd

In [126]:
# Leer las comunas de la Región Metropolitana desde un archivo shapefile y dejar en crs epsg:4326
comunas = gpd.read_file('data\Comunas\comunas.shp').to_crs(epsg=4674)


In [136]:
# Guardar la comuna Maipu en un archivo shapefile solo con su geometría
maipu = comunas[comunas['Comuna'] == 'Maipú'].reset_index(drop=True)
maipu = maipu['geometry']
# Pasar a geodataframe maipu
maipu = gpd.GeoDataFrame(geometry=maipu)


In [108]:
# Leer ISMT de data\ISMT\ISMT_ZC_all_v3_average.shp
ismt = gpd.read_file('data\ISMT\ISMT_ZC_all_v3_average.shp').to_crs(epsg=4674)
# Filtrar por la comuna de Maipu
ismt_maipu = ismt[ismt['nom_cmn'] == 'MAIPÚ'].reset_index(drop=True)
# Me quedo solo con 'prom_ismt', 'AVE_GSE', geocode
ismt_maipu = ismt_maipu[['prom_ismt', 'AVE_GSE', 'geocode']]
ismt_maipu.head(1)


,prom_ismt,AVE_GSE,geocode
0,0.835958,C3,13119201006


In [109]:
# leer data\ZONA_C17\ZONA_C17.shp
zona_c17 = gpd.read_file('data\ZONA_C17\ZONA_C17.shp')
zonas_maipu = zona_c17.loc[zona_c17['NOM_COMUNA']=='MAIPÚ'].reset_index(drop=True) 
zonas_maipu.head(1)


,REGION,NOM_REGION,PROVINCIA,NOM_PROVIN,COMUNA,NOM_COMUNA,URBANO,DISTRITO,LOC_ZON,GEOCODIGO,SHAPE_Leng,SHAPE_Area,geometry
0,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13119,MAIPÚ,MAIPÚ,20,6,13119201006,0.021346,0.000025,"POLYGON ((-70.76309 -33.46351, -70.76309 -33.4..."


In [117]:
# Hacer join entre ismt_maipu y zonas_maipu con geocode y GEOCODIGO
ismt_maipu_zonas = ismt_maipu.merge(zonas_maipu, left_on='geocode', right_on='GEOCODIGO', how='inner')
# Me quedo solo con comuna, LOC_ZON, prom_ismt, AVE_GSE, geoemtry
ismt_maipu_zonas = ismt_maipu_zonas[['NOM_COMUNA', 'LOC_ZON', 'prom_ismt', 'AVE_GSE', 'geometry']]
# Pasar de dataframe a geodataframe ismt_maipu_zonas con crs epsg:4674
ismt_maipu_zonas = gpd.GeoDataFrame(ismt_maipu_zonas, crs=4674)
ismt_maipu_zonas.head(1)


,NOM_COMUNA,LOC_ZON,prom_ismt,AVE_GSE,geometry
0,MAIPÚ,6,0.835958,C3,"POLYGON ((-70.76309 -33.46351, -70.76309 -33.4..."


In [129]:
# Leer data\C2017-2012_RM_pob_socioec\C2017-2012_RM_pob_socioec.shp
pob_socioec = gpd.read_file('data\C2017-2012_RM_pob_socioec\C2017-2012_RM_pob_socioec.shp').to_crs(epsg=4674)
pob_socioec.head(1)

,MANZENT_I,puntaje_es,EDUC,puntaje_au,Hacin,Hacinpunt,NSE,GSE_final,quintil,ID_MANZENT,...,EDAD_15A64,EDAD_6A14,PERSONAS,TOTAL_VIV,index_righ,NOMBRE,TIPO_AREA,SHAPE_Leng,SHAPE_Area,geometry
0,13101071001002,447.369668,12.296209,48,1.057055,120,615.703002,C2,Q1,13101071001002,...,579,79,764,287,385,Santiago,Ciudad,427491.444262,5.821041e+08,"POLYGON ((-70.67695 -33.42691, -70.67685 -33.4..."


In [138]:
# Intersectar maipu con pob_socioec
maipu_pob_socioec = gpd.overlay(pob_socioec, maipu, how='intersection')

In [61]:
# Leer data\manzanas\MANZANA_IND_C17.shp
manzanas = gpd.read_file('data\manzanas\MANZANA_IND_C17.shp')
manzanas_maipu = manzanas.loc[manzanas['NOM_COMUNA']=='MAIPÚ'].reset_index(drop=True)
manzanas_maipu

,REGION,NOM_REGION,PROVINCIA,NOM_PROVIN,COMUNA,NOM_COMUNA,DISTRITO,LOC_ZON,ENT_MAN,CATEGORIA,NOM_CATEGO,MANZENT_I,TOTAL_PERS,TOTAL_VIVI,SHAPE_Leng,SHAPE_Area,geometry
0,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13119,MAIPÚ,20,6,1,1,CD,13119201006001,1515,441,0.037531,7.502504e-06,"POLYGON ((-70.76100 -33.46423, -70.76045 -33.4..."
1,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13119,MAIPÚ,20,6,2,1,CD,13119201006002,71,20,0.002647,3.779597e-07,"POLYGON ((-70.76106 -33.46479, -70.76110 -33.4..."
2,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13119,MAIPÚ,20,6,26,1,CD,13119201006026,178,52,0.003686,5.591587e-07,"POLYGON ((-70.75815 -33.46490, -70.75821 -33.4..."
3,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13119,MAIPÚ,20,6,27,1,CD,13119201006027,109,28,0.002363,3.115026e-07,"POLYGON ((-70.75728 -33.46505, -70.75733 -33.4..."
4,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13119,MAIPÚ,20,7,1,1,CD,13119201007001,288,78,0.004532,6.667583e-07,"POLYGON ((-70.75715 -33.46503, -70.75702 -33.4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4600,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13119,MAIPÚ,22,4,901,1,CD,13119221004901,106,38,0.020557,3.844030e-06,"MULTIPOLYGON (((-70.72528 -33.48180, -70.72513..."
4601,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13119,MAIPÚ,23,1,901,1,CD,13119231001901,59,17,0.014903,2.953232e-06,"MULTIPOLYGON (((-70.78703 -33.56019, -70.78699..."
4602,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13119,MAIPÚ,23,2,901,1,CD,13119231002901,350,106,0.015686,3.619281e-06,"MULTIPOLYGON (((-70.77913 -33.56614, -70.77907..."
4603,13,REGIÓN METROPOLITANA DE SANTIAGO,131,SANTIAGO,13119,MAIPÚ,23,3,901,1,CD,13119231003901,58,32,0.011865,1.715738e-06,"MULTIPOLYGON (((-70.77493 -33.56971, -70.77495..."
